<h4>📥 Load scholarship data from Hugging Face dataset (parquet format)</h4>

In [ ]:
import pandas as pd
from pprint import pprint

# Read scholarship data from parquet file
df = pd.read_parquet("hf://datasets/NetraVerse/indian-govt-scholarships/data/train-00000-of-00001.parquet")

# Select only text and label columns (exclude binary file_name field)
df = df[['label', 'text']]

# Convert to records format
data = df.to_dict('records')

print(f"Loaded {len(data)} scholarship documents")
pprint(data[:2])

<h4>📦 Install required dependencies for vector database, embeddings, and deep learning</h4>

In [ ]:
! pip install qdrant-client
! pip install sentence-transformers
! pip install torch

<h4>🔧 Initialize Qdrant vector database client and SentenceTransformer embedding encoder</h4>

In [ ]:
from qdrant_client import models, QdrantClient
from sentence_transformers import SentenceTransformer

# create the vector database client
qdrant = QdrantClient(":memory:") # Create in-memory Qdrant instance


# Create the embedding encoder
encoder = SentenceTransformer('all-MiniLM-L6-v2') # Model to create embeddings

<h4>🗄️ Create vector collection for storing scholarship embeddings with cosine similarity</h4>

In [ ]:
# Create collection to store the scholarship data
collection_name="scholarships"

qdrant.recreate_collection(
    collection_name=collection_name,
    vectors_config=models.VectorParams(
        size=encoder.get_sentence_embedding_dimension(), # Vector size is defined by used model
        distance=models.Distance.COSINE
    )
)

<h4>⬆️ Generate embeddings for each document and upload to vector database</h4>

In [ ]:
points_to_upload = []
for idx, doc in enumerate(data):
    points_to_upload.append(
        models.PointStruct(
            id=idx,
            vector=encoder.encode(doc["text"]).tolist(),  # Use 'text' field for scholarship data
            payload=doc
        )
    )

# vectorize!
qdrant.upload_points(
    collection_name=collection_name,
    points=points_to_upload
)

In [23]:
# Display first document's text and embedding
first_doc = data[0]
first_text = first_doc['text']
first_vector = encoder.encode(first_text).tolist()

print("DOCUMENT TEXT:")
print(f"Text (first 500 chars): {first_text[:500]}...")
print("EMBEDDING VECTOR:")
print(f"Vector dimension: {len(first_vector)}")
print(f"First 20 values: {first_vector[:20]}")


DOCUMENT TEXT:
Text (first 500 chars): PRAGATI SCHOLARSHIP SCHEME...
EMBEDDING VECTOR:
Vector dimension: 384
First 20 values: [-0.055145297199487686, 0.09013018757104874, -0.06578734517097473, -0.015903353691101074, -0.017773550003767014, -0.02971433289349079, -0.024064399302005768, 0.00018253223970532417, -0.0133947329595685, 0.017595848068594933, 0.06292257457971573, 0.0280141681432724, -0.04938691109418869, -0.004657465498894453, -0.00011058375093853101, -0.0664399191737175, -0.08881905674934387, 0.029758675023913383, 0.009087733924388885, -0.02971675992012024]


<h4>🔢 Example: View first document's text and its embedding vector</h4>

<h4>💬 Define user query for testing the RAG system</h4>

In [ ]:
user_prompt = "Documents required for PRIME MINISTER’S SCHOLARSHIP SCHEME for RPF"

<h4>🔍 Convert user query into embedding vector for semantic search</h4>

In [ ]:
query_vector = encoder.encode(user_prompt).tolist()

<h4>🎯 Search vector database for top 3 most relevant scholarship documents</h4>

In [ ]:
# Search time for awesome wines!
from qdrant_client import QdrantClient
from qdrant_client.models import SearchParams, ScoredPoint

hits = qdrant.query_points(
    collection_name=collection_name,
    query=query_vector,
    limit=3
)

<h4>📄 Display retrieved search results with metadata and similarity scores</h4>

In [ ]:
for hit in hits.points: # Corrected: iterate over hits.points to get the ScoredPoint objects
  pprint(hit)

<h4>🤖 Load TinyLlama model and generate response WITHOUT retrieval context (baseline)</h4>

In [ ]:
# For Hugging Face models
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
from huggingface_hub import login


# Log in to Hugging Face Hub (requires a token set in Colab secrets as 'HF_TOKEN')
# You can get a token from https://huggingface.co/settings/tokens and add it to Colab secrets.
try:
    hf_token =""
    if hf_token:
        login(token=hf_token)
        print("[green]Successfully logged into Hugging Face Hub.")
    else:
        print("Warning: Hugging Face token not found in Colab secrets. Some models might require authentication")
except Exception as e:
    print(f"Error during Hugging Face login: {e}. Some models might not load.")


# Set up device (GPU if available, else CPU)
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

# Load TinyLlama model and tokenizer
model_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name).to(device)

prompt = [
    {"role": "system", "content": "You are a helpful chatbot specializing in Indian government scholarships. Your top priority is to help users find relevant scholarship information and guide them with their queries."},
    {"role": "user","content": user_prompt},
]
inputs = tokenizer.apply_chat_template(
	prompt,
	add_generation_prompt=True,
	tokenize=True,
	return_dict=True,
	return_tensors="pt",
).to(model.device)

outputs = model.generate(**inputs, max_new_tokens=2048)
pprint("Response without RAG and with TinyLlama:")
pprint(tokenizer.decode(outputs[0][inputs["input_ids"].shape[-1]:]))

<h4>📋 Extract payload data from search results for RAG augmentation</h4>

In [ ]:
# define a variable to hold the search results
search_results = [hit.payload for hit in hits.points]

<h4>✨ Generate response WITH retrieval context (RAG-enhanced)</h4>

In [ ]:
# For Hugging Face models
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
from huggingface_hub import login

# Load TinyLlama model and tokenizer
model_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name).to(device)

prompt = [
    {"role": "system", "content": "You are a helpful chatbot specializing in Indian government scholarships. Your top priority is to help users find relevant scholarship information and guide them with their queries."},
    {"role": "user","content": user_prompt},
    {"role": "assistant", "content": str(search_results)},
]
inputs = tokenizer.apply_chat_template(
	prompt,
	add_generation_prompt=True,
	tokenize=True,
	return_dict=True,
	return_tensors="pt",
).to(model.device)
pprint("Response with  RAG and with TinyLlama:")

outputs = model.generate(**inputs, max_new_tokens=250)
pprint(tokenizer.decode(outputs[0][inputs["input_ids"].shape[-1]:]))

<h4>🌐 Launch interactive Gradio chatbot interface with full RAG pipeline</h4>

In [ ]:
import gradio as gr

def scholarship_chatbot(message, history):
    # Encode user query
    query_vector = encoder.encode(message).tolist()
    
    # Search for relevant scholarships
    hits = qdrant.query_points(
        collection_name=collection_name,
        query=query_vector,
        limit=3
    )
    
    search_results = [hit.payload for hit in hits.points]
    
    # Generate response with LLM
    prompt = [
        {"role": "system", "content": "You are a helpful chatbot specializing in Indian government scholarships. Help users find relevant scholarship information based on their queries."},
        {"role": "user", "content": message},
        {"role": "assistant", "content": f"Based on our database: {str(search_results)}"}
    ]
    
    inputs = tokenizer.apply_chat_template(
        prompt,
        add_generation_prompt=True,
        tokenize=True,
        return_dict=True,
        return_tensors="pt",
    ).to(model.device)
    
    outputs = model.generate(**inputs, max_new_tokens=250)
    response = tokenizer.decode(outputs[0][inputs["input_ids"].shape[-1]:])
    
    return response

# Launch Gradio interface
demo = gr.ChatInterface(
    scholarship_chatbot,
    title="🎓 Indian Government Scholarship Chatbot",
    description="Ask me about Indian government scholarships!",
    examples=[
        "What scholarships are available for engineering students?",
        "Tell me about AICTE scholarships",
        "Are there scholarships for women in STEM?"
    ]
)

demo.launch()